# Commerce Challenge 2 - Baseline Submission

This notebook provides a simple baseline for **Commerce Challenge 2: Personalized Product Recommendations**.

**Goal**: Recommend top-10 SKUs for each customer (wide format)
**Metric**: NDCG@10 - Higher is better

## Instructions:
1. **Replace API credentials** in the first cell with your team's API key and name
2. **Run all cells** to generate and submit baseline predictions
3. **Check the output** for your submission score

This baseline uses a simple popularity-based recommender system.


In [ ]:
# 1. Initialize Client and Load Data

import pandas as pd
import numpy as np
from agentds import BenchmarkClient

# 🔑 REPLACE WITH YOUR CREDENTIALS
# 🔑 REPLACE WITH YOUR CREDENTIALS
client = BenchmarkClient(
    api_key="your-api-key-here",        # Get from your team dashboard
    team_name="your-team-name-here"     # Your exact team name
)

# Load data from PVC paths
print("📂 Loading Commerce Challenge 2 data...")

# Load purchase history data
train_purchases = pd.read_csv("../Commerce/purchases_train.csv")
test_purchases = pd.read_csv("../Commerce/purchases_test.csv")

print(f"✅ Data loaded:")
print(f"   Train purchases: {train_purchases.shape}")
print(f"   Test purchases: {test_purchases.shape}")
print(f"   Train columns: {list(train_purchases.columns)}")
print(f"   Test columns: {list(test_purchases.columns)}")


📂 Loading Commerce Challenge 2 data...
✅ Data loaded:
   Train purchases: (16616, 4)
   Test purchases: (500, 2)
   Train columns: ['customer_id', 'order_id', 'month', 'sku_id']
   Test columns: ['customer_id', 'month']


In [ ]:
# 2. Frequency-Based Baseline (Personalized)

# Load products_sub to get valid SKUs for Challenge 2
products_sub = pd.read_csv("/Users/amolighatimma/Desktop/AgentDS/Commerce/products_sub.csv")
valid_skus = set(products_sub['sku_id'].tolist())

# Filter train_purchases to only include valid SKUs
train_filtered = train_purchases[train_purchases['sku_id'].isin(valid_skus)]

print(f"📊 Filtered training data: {train_filtered.shape[0]} purchases for {len(valid_skus)} valid SKUs")

# 1. Per-customer SKU frequency table
freq_tbl = (train_filtered
            .groupby(["customer_id", "sku_id"])
            .size()
            .rename("cnt")
            .reset_index())

# 2. Global popularity for tie-breaking
glob_pop = (train_filtered["sku_id"]
            .value_counts()
            .rename_axis("sku_id")
            .rename("glob_cnt")
            .reset_index())

# Merge global popularity into freq table
freq_tbl = freq_tbl.merge(glob_pop, on="sku_id", how="left")

print(f"✅ Built frequency table: {len(freq_tbl)} customer-SKU pairs")

# Get test customers who need recommendations
test_customers = test_purchases['customer_id'].unique()
print(f"🎯 Need to recommend for {len(test_customers)} customers")

# Create personalized recommendations
recommendations = []

for customer_id in test_customers:
    # Get customer's frequency scores
    customer_freq = freq_tbl[freq_tbl['customer_id'] == customer_id].copy()
    
    if len(customer_freq) > 0:
        # Customer has purchase history - use frequency + global popularity
        customer_freq['score'] = customer_freq['cnt'] + 0.0001 * customer_freq['glob_cnt']
        recommended_skus = customer_freq.nlargest(10, 'score')['sku_id'].tolist()
    else:
        # New customer - use global popularity
        recommended_skus = glob_pop.head(10)['sku_id'].tolist()
    
    # Ensure we have exactly 10 recommendations
    if len(recommended_skus) < 10:
        # Fill remaining with other popular SKUs
        remaining_skus = [sku for sku in glob_pop['sku_id'].tolist() if sku not in recommended_skus]
        recommended_skus.extend(remaining_skus[:10-len(recommended_skus)])
    
    recommended_skus = recommended_skus[:10]
    
    # Create row: customer_id, sku_id_1, sku_id_2, ..., sku_id_10
    row = [customer_id] + recommended_skus
    recommendations.append(row)

# Create submission DataFrame in wide format
columns = ['customer_id'] + [f'sku_id_{i}' for i in range(1, 11)]
submission_df = pd.DataFrame(recommendations, columns=columns)


📊 Filtered training data: 16616 purchases for 750 valid SKUs
✅ Built frequency table: 14462 customer-SKU pairs
🎯 Need to recommend for 500 customers
✅ Predictions saved: 500 customers
   Preview:    customer_id  sku_id_1  sku_id_2  sku_id_3  sku_id_4  sku_id_5  sku_id_6  \
0            1       615       611       632       648       584       570   
1            2        30       443       188       431       405       140   
2            3       387       213        56      1247       138       173   

   sku_id_7  sku_id_8  sku_id_9  sku_id_10  
0       562       595       633        697  
1       529        76       432        467  
2       328      1211       366        321  
   Format: Personalized frequency-based recommendations


In [6]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares

# -----------------------------
# 1. Load Data
# -----------------------------
train_purchases = pd.read_csv("../Commerce/purchases_train.csv")
test_purchases = pd.read_csv("../Commerce/purchases_test.csv")
products_sub = pd.read_csv("/Users/amolighatimma/Desktop/AgentDS/Commerce/products_sub.csv")

valid_skus = set(products_sub['sku_id'].tolist())
train_filtered = train_purchases[train_purchases['sku_id'].isin(valid_skus)]

# -----------------------------
# 2. Map IDs to indices
# -----------------------------
customer_ids = train_filtered['customer_id'].unique()
sku_ids = train_filtered['sku_id'].unique()
customer2idx = {cid: i for i, cid in enumerate(customer_ids)}
sku2idx = {sid: i for i, sid in enumerate(sku_ids)}
idx2sku = {i: sid for sid, i in sku2idx.items()}

# -----------------------------
# 3. Build interaction matrix
# -----------------------------
rows = train_filtered['customer_id'].map(customer2idx)
cols = train_filtered['sku_id'].map(sku2idx)
data = np.ones(len(train_filtered))  # implicit feedback
interaction_matrix = coo_matrix((data, (rows, cols)), shape=(len(customer_ids), len(sku_ids)))
interaction_matrix = interaction_matrix.tocsr()

# -----------------------------
# 4. Train ALS model
# -----------------------------
als_model = AlternatingLeastSquares(
    factors=64,
    regularization=0.1,
    iterations=30,
    use_gpu=False
)
als_model.fit(interaction_matrix.T)  # items x users

# -----------------------------
# 5. Top global SKUs for cold-start
# -----------------------------
top_global_skus = train_filtered['sku_id'].value_counts().head(10).index.tolist()

# -----------------------------
# 6. Generate top-10 recommendations
# -----------------------------
test_customers = test_purchases['customer_id'].unique()
recommendations = []

for cust in test_customers:
    if cust in customer2idx:
        cust_idx = customer2idx[cust]
        
        # Option 2: manual scoring (stable)
        user_vector = als_model.user_factors[cust_idx]       # 1 x latent factors
        scores = als_model.item_factors @ user_vector       # score for each item
        top_idx = np.argsort(-scores)[:10]                  # top-10 indices
        recommended_skus = [idx2sku[i] for i in top_idx]

    else:
        # Cold-start: top global SKUs
        recommended_skus = top_global_skus.copy()

    # Ensure exactly 10 SKUs
    if len(recommended_skus) < 10:
        remaining_skus = [sku for sku in top_global_skus if sku not in recommended_skus]
        recommended_skus.extend(remaining_skus[:10 - len(recommended_skus)])

    recommended_skus = recommended_skus[:10]
    recommendations.append([cust] + recommended_skus)

# -----------------------------
# 7. Create submission CSV
# -----------------------------
columns = ['customer_id'] + [f'sku_id_{i}' for i in range(1, 11)]
submission_df = pd.DataFrame(recommendations, columns=columns)
submission_df.to_csv("commerce_challenge2_predictions.csv", index=False)

print("✅ ALS top-10 recommendations ready (stable)!")
print(submission_df.head())

/Users/amolighatimma/anaconda3/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.0001628398895263672 seconds
  warnings.warn(


  0%|          | 0/30 [00:00<?, ?it/s]

✅ ALS top-10 recommendations ready (stable)!
   customer_id  sku_id_1  sku_id_2  sku_id_3  sku_id_4  sku_id_5  sku_id_6  \
0            1        10      1246        56       907      1233       312   
1            2       533      1233       268       907      1123      1129   
2            3      1233      1061       206       533       268      1155   
3            4       533      1123      1129       268       907        10   
4            5      1129       533        10      1020      1233       496   

   sku_id_7  sku_id_8  sku_id_9  sku_id_10  
0       533      1129       809        974  
1        56       636        10       1246  
2      1020       907      1301       1109  
3       562       312       636       1233  
4        56       333       411       1123  


Below marks the shift from using ALS models to anti-recommendation theory being integrated into the pipeline

In [ ]:
# IDEA 1: IDENTIFY AND REMOVE BAD RECOMMENDATIONS
import pandas as pd
import numpy as np

train_purchases = pd.read_csv("../Commerce/purchases_train.csv")
test_purchases = pd.read_csv("../Commerce/purchases_test.csv")
products_sub = pd.read_csv("/Users/amolighatimma/Desktop/AgentDS/Commerce/products_sub.csv")

valid_skus = set(products_sub['sku_id'].tolist())
train_filtered = train_purchases[train_purchases['sku_id'].isin(valid_skus)]

print("🎯 Anti-recommendation strategy...")

# Your working 0.0769 approach
freq_tbl = (train_filtered.groupby(["customer_id", "sku_id"])
            .size().rename("cnt").reset_index())

glob_pop = (train_filtered["sku_id"].value_counts()
            .rename_axis("sku_id").rename("glob_cnt").reset_index())
glob_pop['glob_score'] = np.log1p(glob_pop['glob_cnt'])

freq_tbl = freq_tbl.merge(glob_pop[['sku_id', 'glob_score']], on='sku_id', how='left')
freq_tbl['glob_score'] = freq_tbl['glob_score'].fillna(0)

# NEW: Identify potentially BAD recommendations
# Items that customers buy once but never again (buyer's remorse)
one_time_purchases = train_filtered.groupby(['customer_id', 'sku_id']).size().reset_index(name='cnt')
one_time_purchases = one_time_purchases[one_time_purchases['cnt'] == 1]

# Items that are rarely bought by the same customer twice
repeat_purchase_rate = train_filtered.groupby('sku_id').agg({
    'customer_id': ['count', 'nunique']
}).reset_index()
repeat_purchase_rate.columns = ['sku_id', 'total_purchases', 'unique_customers']
repeat_purchase_rate['repeat_rate'] = repeat_purchase_rate['total_purchases'] / repeat_purchase_rate['unique_customers']

# Penalize items with low repeat rates (customers don't buy them again)
repeat_purchase_rate['penalty'] = np.where(
    repeat_purchase_rate['repeat_rate'] < 1.1,  # Less than 10% repeat purchases
    0.5,  # Penalty
    1.0   # No penalty
)

freq_tbl = freq_tbl.merge(repeat_purchase_rate[['sku_id', 'penalty']], on='sku_id', how='left')
freq_tbl['penalty'] = freq_tbl['penalty'].fillna(1.0)

test_customers = test_purchases['customer_id'].unique()
recommendations = []

for customer_id in test_customers:
    customer_freq = freq_tbl[freq_tbl['customer_id'] == customer_id].copy()
    
    if len(customer_freq) > 0:
        # Apply penalty to potentially bad recommendations
        customer_freq['score'] = (customer_freq['cnt'] + 0.05 * customer_freq['glob_score']) * customer_freq['penalty']
        recommended_skus = customer_freq.nlargest(10, 'score')['sku_id'].tolist()
    else:
        recommended_skus = glob_pop.head(10)['sku_id'].tolist()
    
    if len(recommended_skus) < 10:
        remaining_skus = [sku for sku in glob_pop['sku_id'].tolist() if sku not in recommended_skus]
        recommended_skus.extend(remaining_skus[:10-len(recommended_skus)])
    
    recommendations.append([customer_id] + recommended_skus[:10])

columns = ['customer_id'] + [f'sku_id_{i}' for i in range(1, 11)]
submission_df = pd.DataFrame(recommendations, columns=columns)
submission_df.to_csv("commerce_challenge2_predictions_ar.csv", index=False)

print(f"✅ Anti-bad predictions saved: {submission_df.shape[0]} customers")

🎯 Anti-recommendation strategy...
✅ Anti-bad predictions saved: 500 customers


In [ ]:
# ENHANCED ANTI-RECOMMENDATION v2
import pandas as pd
import numpy as np

train_purchases = pd.read_csv("../Commerce/purchases_train.csv")
test_purchases = pd.read_csv("../Commerce/purchases_test.csv")
products_sub = pd.read_csv("/Users/amolighatimma/Desktop/AgentDS/Commerce/products_sub.csv")

valid_skus = set(products_sub['sku_id'].tolist())
train_filtered = train_purchases[train_purchases['sku_id'].isin(valid_skus)]

print("🎯 Enhanced anti-recommendation v2...")

# Your working 0.0769 approach
freq_tbl = (train_filtered.groupby(["customer_id", "sku_id"])
            .size().rename("cnt").reset_index())

glob_pop = (train_filtered["sku_id"].value_counts()
            .rename_axis("sku_id").rename("glob_cnt").reset_index())
glob_pop['glob_score'] = np.log1p(glob_pop['glob_cnt'])

freq_tbl = freq_tbl.merge(glob_pop[['sku_id', 'glob_score']], on='sku_id', how='left')
freq_tbl['glob_score'] = freq_tbl['glob_score'].fillna(0)

# ENHANCED BAD RECOMMENDATION DETECTION

# 1. Items with low repeat rates (buyer's remorse)
repeat_purchase_rate = train_filtered.groupby('sku_id').agg({
    'customer_id': ['count', 'nunique']
}).reset_index()
repeat_purchase_rate.columns = ['sku_id', 'total_purchases', 'unique_customers']
repeat_purchase_rate['repeat_rate'] = repeat_purchase_rate['total_purchases'] / repeat_purchase_rate['unique_customers']

# Stronger penalty for low repeat rates
repeat_purchase_rate['penalty_repeat'] = np.where(
    repeat_purchase_rate['repeat_rate'] < 1.05, 0.3,  # Very low repeat = strong penalty
    np.where(repeat_purchase_rate['repeat_rate'] < 1.2, 0.7,  # Low repeat = medium penalty  
    1.0)  # Good repeat rate = no penalty
)

# 2. Items that are "too popular" (might be too generic)
popularity_penalty = np.where(
    glob_pop['glob_cnt'] > glob_pop['glob_cnt'].quantile(0.8),  # Top 20% most popular
    0.8,  # Slight penalty for ultra-popular items
    1.0   # No penalty for regular items
)
glob_pop['penalty_popularity'] = popularity_penalty

# 3. Items that customers quickly stop buying (if we have temporal data)
if 'month' in train_filtered.columns:
    # Find items where customers don't repurchase in subsequent months
    customer_sequences = train_filtered.sort_values(['customer_id', 'month'])
    customer_sequences['next_purchase_month'] = customer_sequences.groupby('customer_id')['month'].shift(-1)
    customer_sequences['months_until_next'] = customer_sequences['next_purchase_month'] - customer_sequences['month']
    
    # Items with long gaps before repurchase (or no repurchase)
    repurchase_gaps = customer_sequences.groupby('sku_id')['months_until_next'].mean().reset_index()
    repurchase_gaps['penalty_gap'] = np.where(
        repurchase_gaps['months_until_next'] > 3, 0.6,  # Long gaps = penalty
        np.where(repurchase_gaps['months_until_next'].isna(), 0.4,  # Never repurchased = strong penalty
        1.0)  # Quick repurchase = no penalty
    )
else:
    repurchase_gaps = pd.DataFrame({'sku_id': [], 'penalty_gap': []})

# Combine all penalties
freq_tbl = freq_tbl.merge(repeat_purchase_rate[['sku_id', 'penalty_repeat']], on='sku_id', how='left')
freq_tbl = freq_tbl.merge(glob_pop[['sku_id', 'penalty_popularity']], on='sku_id', how='left')

if len(repurchase_gaps) > 0:
    freq_tbl = freq_tbl.merge(repurchase_gaps[['sku_id', 'penalty_gap']], on='sku_id', how='left')
    freq_tbl['penalty_gap'] = freq_tbl['penalty_gap'].fillna(0.5)  # Default penalty if no data
else:
    freq_tbl['penalty_gap'] = 1.0

freq_tbl['penalty_repeat'] = freq_tbl['penalty_repeat'].fillna(1.0)
freq_tbl['penalty_popularity'] = freq_tbl['penalty_popularity'].fillna(1.0)

# Combined penalty (multiply all penalties)
freq_tbl['total_penalty'] = freq_tbl['penalty_repeat'] * freq_tbl['penalty_popularity'] * freq_tbl['penalty_gap']

test_customers = test_purchases['customer_id'].unique()
recommendations = []

for customer_id in test_customers:
    customer_freq = freq_tbl[freq_tbl['customer_id'] == customer_id].copy()
    
    if len(customer_freq) > 0:
        # Apply combined penalty to filter out bad recommendations
        customer_freq['score'] = (customer_freq['cnt'] + 0.05 * customer_freq['glob_score']) * customer_freq['total_penalty']
        recommended_skus = customer_freq.nlargest(12, 'score')['sku_id'].tolist()[:10]  # Get top 12, take best 10
    else:
        # For new customers, use items with good repeat rates
        good_global = glob_pop.merge(repeat_purchase_rate[['sku_id', 'penalty_repeat']], on='sku_id')
        good_global = good_global[good_global['penalty_repeat'] > 0.8]  # Only items with good repeat rates
        if len(good_global) >= 10:
            recommended_skus = good_global.nlargest(10, 'glob_score')['sku_id'].tolist()
        else:
            recommended_skus = glob_pop.head(10)['sku_id'].tolist()
    
    if len(recommended_skus) < 10:
        remaining_skus = [sku for sku in glob_pop['sku_id'].tolist() if sku not in recommended_skus]
        recommended_skus.extend(remaining_skus[:10-len(recommended_skus)])
    
    recommendations.append([customer_id] + recommended_skus[:10])

columns = ['customer_id'] + [f'sku_id_{i}' for i in range(1, 11)]
submission_df = pd.DataFrame(recommendations, columns=columns)
submission_df.to_csv("commerce_challenge2_predictions_Ar2.csv", index=False)

print(f"✅ Enhanced anti-bad v2 predictions saved: {submission_df.shape[0]} customers")
print("🎯 Expected: 0.0775 - 0.0785")

🎯 Enhanced anti-recommendation v2...
✅ Enhanced anti-bad v2 predictions saved: 500 customers
🎯 Expected: 0.0775 - 0.0785


Now we are adding a rank based approach and enhanced features. This builds upon the anti-recommendation strategy.

In [ ]:
# ENHANCED RANK-BASED + SMART FEATURES
import pandas as pd
import numpy as np

train_purchases = pd.read_csv("../Commerce/purchases_train.csv")
test_purchases = pd.read_csv("../Commerce/purchases_test.csv")
products_sub = pd.read_csv("/Users/amolighatimma/Desktop/AgentDS/Commerce/products_sub.csv")

valid_skus = set(products_sub['sku_id'].tolist())
train_filtered = train_purchases[train_purchases['sku_id'].isin(valid_skus)]

print("🚀 Enhanced Rank-Based for 0.085 target...")

# Your working frequency approach
freq_tbl = (train_filtered.groupby(["customer_id", "sku_id"])
            .size().rename("cnt").reset_index())

# Enhanced global popularity with multiple rank-based strategies
glob_pop = (train_filtered["sku_id"].value_counts()
            .rename_axis("sku_id").rename("glob_cnt").reset_index())

# Multiple rank-based scoring strategies
glob_pop['rank'] = glob_pop['glob_cnt'].rank(method='dense', ascending=False)
glob_pop['score_rank_inv_sqrt'] = 1.0 / np.sqrt(glob_pop['rank'])  # Your working one
glob_pop['score_rank_inv_log'] = 1.0 / np.log(glob_pop['rank'] + 1)  # Alternative
glob_pop['score_rank_power'] = 1.0 / (glob_pop['rank'] ** 0.7)  # Power law

# Blend the best performing rank-based scores
glob_pop['glob_score'] = (
    glob_pop['score_rank_inv_sqrt'] * 0.6 + 
    glob_pop['score_rank_inv_log'] * 0.3 +
    glob_pop['score_rank_power'] * 0.1
)

# SMART FEATURE 1: Customer purchase patterns
customer_stats = train_filtered.groupby('customer_id').agg({
    'sku_id': ['count', 'nunique']
}).reset_index()
customer_stats.columns = ['customer_id', 'total_purchases', 'unique_products']
customer_stats['repeat_ratio'] = customer_stats['total_purchases'] / customer_stats['unique_products']

# SMART FEATURE 2: Item quality (repeat purchase rate)
item_quality = train_filtered.groupby('sku_id').agg({
    'customer_id': ['count', 'nunique']
}).reset_index()
item_quality.columns = ['sku_id', 'total_purchases', 'unique_customers']
item_quality['repeat_rate'] = item_quality['total_purchases'] / item_quality['unique_customers']
item_quality['quality_score'] = np.sqrt(item_quality['total_purchases']) * item_quality['repeat_rate']

# SMART FEATURE 3: Simple co-purchase (fast version)
print("🔄 Fast co-purchase analysis...")
customer_items = train_filtered.groupby('customer_id')['sku_id'].apply(list)
multi_purchase = customer_items[customer_items.apply(len) > 1]

co_purchase_counts = {}
for items in multi_purchase:
    for i in range(len(items)):
        for j in range(i+1, len(items)):
            pair = (items[i], items[j])
            co_purchase_counts[pair] = co_purchase_counts.get(pair, 0) + 1

# Convert to item->item mapping
co_purchase_map = {}
for (item1, item2), count in co_purchase_counts.items():
    if item1 not in co_purchase_map:
        co_purchase_map[item1] = {}
    if item2 not in co_purchase_map:
        co_purchase_map[item2] = {}
    co_purchase_map[item1][item2] = count
    co_purchase_map[item2][item1] = count

# Merge all features
freq_tbl = freq_tbl.merge(customer_stats, on='customer_id')
freq_tbl = freq_tbl.merge(glob_pop[['sku_id', 'glob_score']], on='sku_id', how='left')
freq_tbl = freq_tbl.merge(item_quality[['sku_id', 'quality_score']], on='sku_id', how='left')
freq_tbl['glob_score'] = freq_tbl['glob_score'].fillna(0)
freq_tbl['quality_score'] = freq_tbl['quality_score'].fillna(0)

test_customers = test_purchases['customer_id'].unique()
recommendations = []

for customer_id in test_customers:
    customer_freq = freq_tbl[freq_tbl['customer_id'] == customer_id].copy()
    customer_items = customer_freq['sku_id'].tolist()
    
    if len(customer_freq) > 0:
        total_purchases = customer_freq['total_purchases'].iloc[0]
        repeat_ratio = customer_freq['repeat_ratio'].iloc[0]
        
        # ENHANCED SCORING with customer segmentation
        if total_purchases >= 15:
            # Super users: Strong personalization + quality focus
            customer_freq['score'] = (
                customer_freq['cnt'] * 2.5 +           # Very strong personal
                customer_freq['quality_score'] * 0.8 + # High quality focus
                customer_freq['glob_score'] * 0.02     # Minimal global
            )
        elif total_purchases >= 8:
            # Heavy users: Balanced with quality
            customer_freq['score'] = (
                customer_freq['cnt'] * 2.0 +
                customer_freq['quality_score'] * 0.5 +
                customer_freq['glob_score'] * 0.05
            )
        elif total_purchases >= 3:
            # Medium users: Standard
            customer_freq['score'] = (
                customer_freq['cnt'] * 1.5 +
                customer_freq['quality_score'] * 0.3 +
                customer_freq['glob_score'] * 0.08
            )
        else:
            # Light users: More global + quality
            customer_freq['score'] = (
                customer_freq['cnt'] * 1.2 +
                customer_freq['quality_score'] * 0.2 +
                customer_freq['glob_score'] * 0.12
            )
        
        # ADD CO-PURCHASE BOOST (limited to avoid overfitting)
        co_boost_added = 0
        for item in customer_items[:5]:  # Only top 5 purchased items
            if item in co_purchase_map and co_boost_added < 3:  # Limit to 3 boosts
                for co_item, count in list(co_purchase_map[item].items())[:3]:  # Top 3 co-purchases
                    if co_item not in customer_items and count >= 2:  # Only strong co-purchases
                        if co_item in customer_freq['sku_id'].values:
                            customer_freq.loc[customer_freq['sku_id'] == co_item, 'score'] += count * 0.15
                        else:
                            # Only add if it's a strong co-purchase and we have room
                            if co_boost_added < 2:
                                new_score = count * 0.15
                                # Add quality and global if available
                                quality = item_quality[item_quality['sku_id'] == co_item]['quality_score'].iloc[0] if len(item_quality[item_quality['sku_id'] == co_item]) > 0 else 0
                                global_score = glob_pop[glob_pop['sku_id'] == co_item]['glob_score'].iloc[0] if len(glob_pop[glob_pop['sku_id'] == co_item]) > 0 else 0
                                new_score += quality * 0.2 + global_score * 0.05
                                
                                new_row = {
                                    'customer_id': customer_id, 
                                    'sku_id': co_item, 
                                    'cnt': 0,
                                    'total_purchases': total_purchases,
                                    'unique_products': customer_freq['unique_products'].iloc[0],
                                    'repeat_ratio': repeat_ratio,
                                    'glob_score': global_score,
                                    'quality_score': quality,
                                    'score': new_score
                                }
                                customer_freq = pd.concat([customer_freq, pd.DataFrame([new_row])], ignore_index=True)
                                co_boost_added += 1
        
        recommended_skus = customer_freq.nlargest(12, 'score')['sku_id'].tolist()[:10]
        
    else:
        # New customer: High-quality global items
        high_quality_global = glob_pop.merge(item_quality[['sku_id', 'quality_score']], on='sku_id')
        high_quality_global['score'] = (
            high_quality_global['glob_score'] * 1.2 + 
            high_quality_global['quality_score'] * 0.8
        )
        recommended_skus = high_quality_global.nlargest(10, 'score')['sku_id'].tolist()
    
    # Ensure exactly 10
    if len(recommended_skus) < 10:
        remaining_skus = [sku for sku in glob_pop['sku_id'].tolist() if sku not in recommended_skus]
        recommended_skus.extend(remaining_skus[:10-len(recommended_skus)])
    
    recommendations.append([customer_id] + recommended_skus[:10])

columns = ['customer_id'] + [f'sku_id_{i}' for i in range(1, 11)]
submission_df = pd.DataFrame(recommendations, columns=columns)
submission_df.to_csv("commerce_challenge2_predictions_er.csv", index=False)

print(f"✅ Enhanced rank predictions saved: {submission_df.shape[0]} customers")
print("🎯 Expected: 0.079 - 0.082")

🚀 Enhanced Rank-Based for 0.085 target...
🔄 Fast co-purchase analysis...
✅ Enhanced rank predictions saved: 500 customers
🎯 Expected: 0.079 - 0.082


To push up a few points in our metric, we tried to ensure data leakage was accounted for producing the best model.

In [161]:
# DATA LEAKAGE APPROACH - FIND HIDDEN PATTERNS
import pandas as pd
import numpy as np
from collections import Counter

train_purchases = pd.read_csv("../Commerce/purchases_train.csv")
test_purchases = pd.read_csv("../Commerce/purchases_test.csv")
products_sub = pd.read_csv("/Users/amolighatimma/Desktop/AgentDS/Commerce/products_sub.csv")

valid_skus = set(products_sub['sku_id'].tolist())
train_filtered = train_purchases[train_purchases['sku_id'].isin(valid_skus)]

print("🔍 Data Leakage Approach - Finding Hidden Patterns...")

# Let's analyze what makes the 0.0802 model work
print("Analyzing successful patterns...")

# Strategy 1: Look for customers who behave similarly in train and test
train_customers = set(train_filtered['customer_id'])
test_customers_list = test_purchases['customer_id'].unique()

overlap_customers = [c for c in test_customers_list if c in train_customers]
new_customers = [c for c in test_customers_list if c not in train_customers]

print(f"Overlap customers: {len(overlap_customers)}, New customers: {len(new_customers)}")

# Strategy 2: Find items that are "universal winners" - bought by many different customer types
customer_item_matrix = train_filtered.groupby(['customer_id', 'sku_id']).size().unstack(fill_value=0)
item_customer_count = (customer_item_matrix > 0).sum(axis=0)
universal_items = item_customer_count[item_customer_count >= item_customer_count.quantile(0.8)].index.tolist()

print(f"Universal items (bought by many customers): {len(universal_items)}")

# Strategy 3: Find items with high "conversion rates" - once bought, often bought again
item_stats = train_filtered.groupby('sku_id').agg({
    'customer_id': ['count', 'nunique']
}).reset_index()
item_stats.columns = ['sku_id', 'total_purchases', 'unique_customers']
item_stats['repeat_rate'] = item_stats['total_purchases'] / item_stats['unique_customers']
high_conversion_items = item_stats[item_stats['repeat_rate'] >= 1.5]['sku_id'].tolist()

print(f"High conversion items: {len(high_conversion_items)}")

# Build recommendations using these insights
freq_tbl = (train_filtered.groupby(["customer_id", "sku_id"])
            .size().rename("cnt").reset_index())

glob_pop = (train_filtered["sku_id"].value_counts()
            .rename_axis("sku_id").rename("glob_cnt").reset_index())
glob_pop['rank'] = glob_pop['glob_cnt'].rank(method='dense', ascending=False)
glob_pop['glob_score'] = 1.0 / np.sqrt(glob_pop['rank'])

# Add our new insights
freq_tbl = freq_tbl.merge(glob_pop[['sku_id', 'glob_score']], on='sku_id', how='left')
freq_tbl['glob_score'] = freq_tbl['glob_score'].fillna(0)

# Mark universal and high-conversion items
freq_tbl['is_universal'] = freq_tbl['sku_id'].isin(universal_items)
freq_tbl['is_high_conversion'] = freq_tbl['sku_id'].isin(high_conversion_items)

test_customers = test_purchases['customer_id'].unique()
recommendations = []

for customer_id in test_customers:
    customer_freq = freq_tbl[freq_tbl['customer_id'] == customer_id].copy()
    
    if len(customer_freq) > 0:
        # INNOVATIVE SCORING: Leverage our new insights
        customer_freq['score'] = (
            customer_freq['cnt'] * 2.0 +                    # Personal history
            customer_freq['glob_score'] * 0.08 +           # Global rank
            customer_freq['is_universal'] * 0.5 +          # Universal items boost
            customer_freq['is_high_conversion'] * 0.7      # High conversion boost
        )
        
        # For customers with purchase history, also consider what similar customers bought
        customer_items = customer_freq['sku_id'].tolist()
        
        # Find customers with similar purchase patterns
        similar_customers = train_filtered[
            train_filtered['sku_id'].isin(customer_items) & 
            (train_filtered['customer_id'] != customer_id)
        ]['customer_id'].unique()
        
        if len(similar_customers) > 0:
            similar_purchases = train_filtered[
                train_filtered['customer_id'].isin(similar_customers) & 
                (~train_filtered['sku_id'].isin(customer_items))
            ]
            
            if len(similar_purchases) > 0:
                similar_items = similar_purchases['sku_id'].value_counts().head(3)
                for item, count in similar_items.items():
                    if item not in customer_freq['sku_id'].values and item in valid_skus:
                        # Check if this is a universal or high-conversion item
                        is_uni = item in universal_items
                        is_conv = item in high_conversion_items
                        boost = count * 0.3 + is_uni * 0.3 + is_conv * 0.4
                        
                        new_row = {
                            'customer_id': customer_id, 
                            'sku_id': item, 
                            'cnt': 0,
                            'glob_score': glob_pop[glob_pop['sku_id'] == item]['glob_score'].iloc[0] if len(glob_pop[glob_pop['sku_id'] == item]) > 0 else 0,
                            'is_universal': is_uni,
                            'is_high_conversion': is_conv,
                            'score': boost
                        }
                        customer_freq = pd.concat([customer_freq, pd.DataFrame([new_row])], ignore_index=True)
        
        recommended_skus = customer_freq.nlargest(10, 'score')['sku_id'].tolist()
    else:
        # New customer strategy: focus on universal + high conversion items
        new_cust_scores = glob_pop.copy()
        new_cust_scores['is_universal'] = new_cust_scores['sku_id'].isin(universal_items)
        new_cust_scores['is_high_conversion'] = new_cust_scores['sku_id'].isin(high_conversion_items)
        new_cust_scores['score'] = (
            new_cust_scores['glob_score'] * 0.6 +
            new_cust_scores['is_universal'] * 0.8 +
            new_cust_scores['is_high_conversion'] * 1.0
        )
        recommended_skus = new_cust_scores.nlargest(10, 'score')['sku_id'].tolist()
    
    # Ensure exactly 10
    if len(recommended_skus) < 10:
        remaining_skus = [sku for sku in glob_pop['sku_id'].tolist() if sku not in recommended_skus]
        recommended_skus.extend(remaining_skus[:10-len(recommended_skus)])
    
    recommendations.append([customer_id] + recommended_skus[:10])

columns = ['customer_id'] + [f'sku_id_{i}' for i in range(1, 11)]
submission_df = pd.DataFrame(recommendations, columns=columns)
submission_df.to_csv("commerce_challenge2_predictions.csv", index=False)

print(f"✅ Data leakage approach saved: {submission_df.shape[0]} customers")
print("🎯 This might find patterns others are missing")

🔍 Data Leakage Approach - Finding Hidden Patterns...
Analyzing successful patterns...
Overlap customers: 500, New customers: 0
Universal items (bought by many customers): 164
High conversion items: 13
✅ Data leakage approach saved: 500 customers
🎯 This might find patterns others are missing


Note that the below existing submit prediction code was run after trying a model that was not the best. However, if you run the documented code directly above(data leakage approach), it should have a NDCG of 0.804.

In [170]:
# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Commerce", 2, "commerce_challenge2_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. Move on to Commerce Challenge 3!")


🚀 Submitting predictions...
✅ Prediction submitted successfully!
📊 Score: 0.0800 (NDCG@10)
✅ Validation passed
✅ Submission successful!
   📊 Score: 0.0800
   📏 Metric: NDCG@10
   ✔️  Validation: Passed

🎯 Next steps:
   1. Try incorporating relevant information outside this table!
   2. Move on to Commerce Challenge 3!
